In [1]:
import os
import utils, segmentation, presentation
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

In [2]:
ds_train = segmentation.SegmentationDataset('./cut_stenoses_data/train/', 2)
ds_test = segmentation.SegmentationDataset('./cut_stenoses_data/test/', 2)

batch_size = 8
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [3]:
len(dl_train), len(dl_test)

(74, 19)

In [4]:
device = torch.device('cuda:1')

In [5]:
model = presentation.create_model().to(device)

In [6]:
optimizer = presentation.create_optimizer(model)

In [7]:
# w = 0.
# for imgs, masks in dl_train:
#     w += masks.sum() / (masks.numel() * len(dl_train))
# loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1 / (1-w), 1/ w])).to(device)
loss_fn = nn.CrossEntropyLoss()

In [8]:
for epoch in range(50):
    trn_loss = utils.train(dl_train, loss_fn, model, optimizer)
    val_loss, val_IoUs = segmentation.evaluate(dl_test, loss_fn, model)
    print(epoch, f'{trn_loss:.4f}', f'{val_loss:.4f}', val_IoUs)

0 0.0259 0.0160 tensor([0.])
1 0.0064 0.0142 tensor([0.])
2 0.0055 0.0140 tensor([0.])
3 0.0049 0.0113 tensor([0.4794])
4 0.0045 0.0156 tensor([0.3883])
5 0.0041 0.0134 tensor([0.4685])
6 0.0038 0.0122 tensor([0.5148])
7 0.0035 0.0146 tensor([0.4791])
8 0.0033 0.0141 tensor([0.5053])
9 0.0031 0.0147 tensor([0.4994])
10 0.0029 0.0135 tensor([0.5317])
11 0.0027 0.0137 tensor([0.5394])
12 0.0026 0.0155 tensor([0.5234])
13 0.0025 0.0117 tensor([0.5745])
14 0.0024 0.0138 tensor([0.5499])
15 0.0022 0.0140 tensor([0.5475])
16 0.0022 0.0130 tensor([0.5689])
17 0.0021 0.0141 tensor([0.5580])
18 0.0021 0.0130 tensor([0.5765])
19 0.0019 0.0139 tensor([0.5622])
20 0.0018 0.0132 tensor([0.5748])
21 0.0018 0.0129 tensor([0.5824])
22 0.0018 0.0118 tensor([0.6150])
23 0.0017 0.0138 tensor([0.5713])
24 0.0016 0.0139 tensor([0.5765])
25 0.0016 0.0136 tensor([0.5788])
26 0.0015 0.0133 tensor([0.5913])
27 0.0015 0.0121 tensor([0.6054])
28 0.0014 0.0131 tensor([0.5934])
29 0.0014 0.0134 tensor([0.5918])
30